# import

In [1]:
import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
# import importlib
# importlib.reload(hoge)
from my_library.horse import *

# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_mac2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_mac = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'

# load data

In [90]:
results = load_csv(path_mac+'results.csv')
horse_results = load_csv(path_mac+'horse_results.csv')
peds = load_csv(path_mac+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_mac+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [ ]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 29,
                'learning_rate': 0.021772987388503244,
                'num_iterations': 93,
                'num_leaves': 40
            }

In [92]:
ll = LearnLGBM(peds,results,horse_results)
ll.learn_lgb()


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.080778 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46188
[LightGBM] [Info] Number of data points in the train set: 87521, number of used features: 172


In [95]:
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
ll.learn_lgb2(train,lgbm_params=lgbm_params)

/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.068078 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46188
[LightGBM] [Info] Number of data points in the train set: 87521, number of used features: 172


In [ ]:
# ll = LearnLGBM(peds,results,horse_results)
# ll.get_train_data()


val_x = ll.x_test.iloc[:-10000]
val_y = ll.y_test.iloc[:-10000]
test_query = val_x.groupby(val_x.index).size()
train_query = ll.x_train.groupby(ll.x_train.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

# 正解ラベル　変えた

In [ ]:
def split_data(df, test_size=0.2, rank_learning=True):
    """
    データを学習データと, 訓練データに分ける関数
    """
    df_ = df.copy()
    df_['rank'] = df_['rank'].map(lambda x: int((1/x)*10) if x<4 else(0))
        
    sorted_id_list = df_.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df_.loc[train_id_list]#.drop(['date'], axis=1)
    test = df_.loc[test_id_list]#.drop(['date'], axis=1)
    return train, test

    
train, test = split_data(ll.r.data_c.fillna(0),test_size=0.2,rank_learning=False)
x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_test = test.drop(['rank', 'date','単勝'], axis=1)
y_test = test['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
val_y = y_test.iloc[:-10000]
test_query = x_test.groupby(x_test.index).size()
valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

In [94]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'num_iterations': 97,
                'num_leaves': 42
            }

{'lambdarank_truncation_level': 10,
 'learning_rate': 0.02273417953255777,
 'num_iterations': 97,
 'num_leaves': 42}

{'lambdarank_truncation_level': 10,
 'learning_rate': 0.02273417953255777,
 'num_iterations': 97,
 'num_leaves': 42}

In [ ]:
range(1,50)[28]

In [ ]:
# {'lambdarank_truncation_level': range(1,50)[33], 'learning_rate': 0.09608503291165846, 'num_iterations': range(50,120)[14], 'num_leaves': range(20,50)[2]}

{'lambdarank_truncation_level': 28, 'learning_rate': 0.021772987388503244}
{'lambdarank_truncation_level':  range(1,50)[28], 'learning_rate': 0.02273417953255777, 'num_iterations': range(50,120)[47], 'num_leaves': range(20,50)[22]}

In [ ]:
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [ ]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        # 'num_iterations': hp.choice('num_iterations',range(50,120)),
        # 'num_leaves': hp.choice('num_leaves',range(20,50)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 25      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [ ]:
trials = Trials()
optimize(trials)

# Simulate


In [96]:

start_time = time.time()
for odds in [1.1,2.0,3.0,4.0,5.0,6.0,7.0]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    sl.calc_tansho(ll.r.data_c.iloc[-10000:],return_tables,odds=odds)
    print("time", time.time() - start_time)




odds 1.1
not_bet_count 0
---------------------
単勝
的中率 : 153 / 727
的中% : 21.05 %
収支   : -8650.0 円
的中レース : {'01': 19, '02': 17, '03': 13, '04': 12, '05': 12, '06': 12, '07': 20, '08': 9, '09': 6, '10': 11, '11': 9, '12': 13}
time 22.824225187301636

odds 2.0
not_bet_count 92
---------------------
単勝
的中率 : 115 / 635
的中% : 18.11 %
収支   : -5400.0 円
的中レース : {'01': 19, '02': 17, '03': 13, '04': 12, '05': 12, '06': 12, '07': 20, '08': 9, '09': 6, '10': 11, '11': 9, '12': 13}
time 48.00784182548523

odds 3.0
not_bet_count 236
---------------------
単勝
的中率 : 62 / 491
的中% : 12.63 %
収支   : -3930.0 円
的中レース : {'01': 19, '02': 17, '03': 13, '04': 12, '05': 12, '06': 12, '07': 20, '08': 9, '09': 6, '10': 11, '11': 9, '12': 13}
time 68.47328901290894

odds 4.0
not_bet_count 372
---------------------
単勝
的中率 : 37 / 355
的中% : 10.42 %
収支   : 1110.0 円
的中レース : {'01': 19, '02': 17, '03': 13, '04': 12, '05': 12, '06': 12, '07': 20, '08': 9, '09': 6, '10': 11, '11': 9, '12': 13}
time 89.69175505638123

odds 5.0

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

# race_id_list

In [97]:

202205021011
race_id_list = ['2022050211{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022050209{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022070307{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022070306{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022040107{}'.format(str(i).zfill(2)) for i in range(1,13)]
# race_id_list += ['2022040106{}'.format(str(i).zfill(2)) for i in range(1,13)]



In [ ]:
    
class Predictor(LearnLGBM):


    def __init__(self,peds,results,horse_results,race_id_list):
        super(Predictor, self).__init__(peds,results,horse_results)
        self.race_id_list = race_id_list


    def process_data(self):
        race_id_list = self.race_id_list.copy()
        data =  ShutubaTable.scrape(race_id_list, self.date)
        self.data = data
        peds = self.peds.copy()
        results = self.results.copy()
        horse_results = self.horse_results.copy()


        nopeds_id_list = []
        for ind in data['horse_id'].astype(int).unique():
            if ind not in peds.index:
                nopeds_id_list.append(str(ind))

        if len(nopeds_id_list)!=0:
            peds_tmp = Peds.scrape(nopeds_id_list)
            pe_tmp = Peds(peds_tmp)
            pe_tmp.regularize_peds() 
            new_peds = update_data(peds, pe_tmp.peds_re)
        else:
            new_peds = peds.copy()
        self.peds = new_peds.copy()
        path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
        new_peds.to_csv(path_ft,header=False,index=False,sep=',')
        self.learn_model_ft(path_ft=path_ft)
        model_ft = self.get_model_ft()

        pe = Peds(new_peds)
        pe.regularize_peds()
        pe.vectorize(pe.peds_re,model_ft)
        self.pe = pe
        print("pe finish")
        print("pe regularizrd")

        r = Results(results)
        r.preprocessing()
        horse_id_list = data['horse_id'].astype(str).unique()
        horse_results_tmp = HorseResults.scrape(horse_id_list)
        new_horse_results = update_data(horse_results,horse_results_tmp)
        self.hores_results = new_horse_results.copy()
        hr = HorseResults(new_horse_results)
        self.hr = hr
        print("hr finish")

        r.merge_horse_results(hr)
        r.merge_peds(pe.peds_vec)
        r.process_categorical()  
        self.r = r
    
    
    def predict(self, race_id):
        data =  ShutubaTable.scrape([str(race_id)], self.date)
        st = ShutubaTable(data)
        st.preprocessing()
        st.merge_horse_results(self.hr)
        st.merge_peds(self.pe.peds_vec)
        st.process_categorical(self.r.le_horse, self.r.le_jockey, self.r.data_pe)
        self.st = st
        sl = RankSimulater(self.model)
        pred_table = sl.return_pred_table(st.data_c)
        self.sl = sl
        print(pred_table)
            

    def show_results_today(self):
        data =  ShutubaTable.scrape(self.race_id_list, self.date)
        self.data = data.copy()
        st = ShutubaTable(self.data)
        st.preprocessing()
        st.merge_horse_results(self.hr)
        st.merge_peds(self.pe.peds_vec)
        st.process_categorical(self.r.le_horse, self.r.le_jockey, self.r.data_pe)
        sl = RankSimulater(self.model)
        sl.return_table_today(self.race_id_list)
        sl.show_results_today(st ,self.race_id_list)

In [98]:
pt = Predictor(peds,results,horse_results,race_id_list)
pt.learn_lgb(lgbm_params=lgbm_params)


pe finish
pe regularizrd


[Errno 54] Connection reset by peer

hr finish


/Users/rince/Library/Python/3.7/lib/python/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.145745 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46172
[LightGBM] [Info] Number of data points in the train set: 87521, number of used features: 172


In [130]:
pt_model = pt.model
ll_model = ll.model


# 東京

In [123]:
base  = '2022050211'
pt.predict(base+'07')


              馬番    scores
202205021107   1 -0.937222
202205021107   2 -0.937222
202205021107   3 -0.937222
202205021107   4 -0.937222
202205021107   5 -0.937222


# 中京

In [126]:
base  = '2022070307'
for i in ['07','08','09','10','11']:
    pt.predict(base+i)


              馬番    scores
202207030707   2  0.696487
202207030707   9 -0.046694
202207030707   4 -0.578141
202207030707   3 -0.645620
202207030707   7 -0.959719
202207030707   1 -1.013527
202207030707   6 -1.013527
202207030707   8 -1.013527
202207030707   5 -1.040436



              馬番    scores
202207030708  10  0.562001
202207030708   6 -0.246847
202207030708   3 -0.349187
202207030708  11 -0.961065
202207030708   2 -0.962900
202207030708   8 -0.965341
202207030708   1 -0.979633
202207030708   5 -0.983140
202207030708   7 -1.009615
202207030708   4 -1.013527
202207030708   9 -1.013527



              馬番    scores
202207030709   9  1.010790
202207030709   1  0.856161
202207030709   7  0.677173
202207030709   2  0.353756
202207030709   8  0.048342
202207030709   5 -0.160111
202207030709   3 -0.506173
202207030709   4 -0.841543
202207030709   6 -0.973623



              馬番    scores
202207030710  12  1.199838
202207030710   8  0.937684
202207030710   2  0.475729
202207030710  13  0.469457
202207030710  15  0.090732
202207030710  14 -0.228860
202207030710   6 -0.358393
202207030710  11 -0.440840
202207030710   4 -0.666071
202207030710  10 -0.713065
202207030710  16 -0.762253
202207030710   1 -0.803113
202207030710   9 -0.877766
202207030710   5 -0.913407
202207030710   3 -1.013527
202207030710   7 -1.013527



              馬番    scores
202207030711   7  0.796032
202207030711   9 -0.020983
202207030711  15 -0.231673
202207030711   5 -0.476024
202207030711  11 -0.937222
202207030711  16 -0.937222
202207030711  14 -0.937222
202207030711  13 -0.937222
202207030711  12 -0.937222
202207030711   1 -0.937222
202207030711  10 -0.937222
202207030711   2 -0.937222
202207030711   8 -0.937222
202207030711   6 -0.937222
202207030711   4 -0.937222
202207030711   3 -0.937222
202207030711  17 -0.937222


# 新潟

In [127]:
base  = '2022040107'
for i in ['07','08','09','10','11']:
    pt.predict(base+i)


              馬番    scores
202204010707  15 -0.346881
202204010707  16 -0.573098
202204010707   1 -0.937222
202204010707   2 -0.937222
202204010707   3 -0.937222
202204010707   4 -0.937222
202204010707   5 -0.937222
202204010707   6 -0.937222
202204010707   7 -0.937222
202204010707   8 -0.937222
202204010707   9 -0.937222
202204010707  10 -0.937222
202204010707  11 -0.937222
202204010707  12 -0.937222
202204010707  13 -0.937222
202204010707  14 -1.387775



              馬番    scores
202204010708   6 -0.266327
202204010708  13 -0.318116
202204010708   5 -0.455465
202204010708   4 -0.475451
202204010708  12 -0.510336
202204010708   3 -0.571341
202204010708   2 -0.791436
202204010708   7 -1.169820
202204010708   1 -1.233953
202204010708   8 -1.289030
202204010708   9 -1.328527
202204010708  11 -1.516580
202204010708  10 -1.695328



              馬番    scores
202204010709   7 -0.166273
202204010709  16 -0.320507
202204010709  12 -0.405103
202204010709   6 -0.447879
202204010709   4 -0.451738
202204010709   9 -0.517426
202204010709  10 -0.657550
202204010709   1 -0.681184
202204010709  13 -0.701236
202204010709  11 -0.811603
202204010709  14 -0.956243
202204010709  15 -1.033229
202204010709   5 -1.046215
202204010709   2 -1.154096
202204010709   3 -1.179329
202204010709   8 -1.275510



              馬番    scores
202204010710  12  0.014143
202204010710  10 -0.217982
202204010710   8 -0.270307
202204010710  13 -0.332014
202204010710   6 -0.345817
202204010710   7 -0.464156
202204010710  14 -0.472230
202204010710   2 -0.846489
202204010710   4 -1.066491
202204010710   1 -1.082784
202204010710  11 -1.183934
202204010710   9 -1.190904
202204010710   3 -1.224308
202204010710   5 -1.265969



              馬番    scores
202204010711   8  0.655819
202204010711   2  0.597485
202204010711  12  0.008215
202204010711   3 -0.042368
202204010711  11 -0.259554
202204010711  10 -0.366296
202204010711   4 -0.477588
202204010711   5 -0.586729
202204010711   1 -0.706779
202204010711   9 -1.067775
202204010711   6 -1.407519
202204010711   7 -1.818734


In [132]:
pt.model = pt_model
pt.show_results_today()

# 日付に注意

In [3]:
date = '2022/12/31'

# Results scraping

In [5]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

# Horse_results scraping

In [6]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)
# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

# Peds scraping

In [7]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

# Return scraping

In [52]:
returns_tmp = Return.scrape(race_id_list)


# update

In [53]:
new_results = update_data(load_csv(path_mac+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_mac+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_mac+'peds.csv'),peds_tmp)
returns = load_csv(path_mac+'return.csv').rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
returns_tmp = returns_tmp.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)
new_return = update_data(returns,returns_tmp)


# save

In [89]:
new_results.to_csv(path_mac+'results.csv')
new_horse_results.to_csv(path_mac+'horse_results.csv')
new_peds.to_csv(path_mac+'peds.csv')
new_return.to_csv(path_mac+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

# rank = [int((1/i)*10)] と変換してみてはどうか